In [2]:
import sys
sys.path.append('data_process_codes')
sys.path.append('result_process_codes')
sys.path.append('raw_datas')
sys.path.append('output_dir')
import datetime as dt
import numpy as np
import pandas as pd

from result_process_codes.daixiao_comp_basic_info import daixiao_comp_basic_info
#全局变量设置
##数据路径
path_基本数据 = r'.\raw_datas\基础数据.xlsx'
path_代销数据 = r'.\raw_datas\代销数据0526.xlsx'
path_净值数据 = r'.\raw_datas\周度净值_均值处理_未筛选存续期.xlsx'
path_代销机构数据 = r'.\raw_datas\wind_bank_info_230628.csv'
path_现金管理类数据 = r'.\raw_datas\现金管理类_七日年化.xlsx'
path_all净值数据 = r'.\raw_datas\py_all_net_value_0704.csv'
path_周度日期 = r'.\raw_datas\周度日期20220101-20230630.xlsx'
path_outputdir = r'.\output_dir'

##常规数据时间区间
start_date = dt.datetime(2023,5,1)#需要更新的数据起始时间
end_date = dt.datetime(2023,5,31)#需要更新的数据结束时间（一般为最新时间）
##时序数据时间区间
start_date_ts = dt.datetime(2023,1,1)#需要更新的数据起始时间
end_date_ts = dt.datetime(2023,5,31)#需要更新的数据结束时间（一般为最新时间）



In [3]:
df1 = pd.read_excel(path_基本数据,index_col=0).infer_objects()
df1['理财公司简称'] = df1.apply(lambda x:x[0].split('有限')[0],axis=1)
df2 = pd.read_excel(path_代销数据)
df_date = pd.read_excel(path_周度日期)
df7 = pd.read_excel(path_净值数据,index_col=0).fillna(method='ffill',axis=1)
df8 = pd.read_csv(path_代销机构数据,encoding='utf-8').drop_duplicates(subset='comp_simple_name', keep='first', inplace=False)
df9 = pd.read_excel(path_现金管理类数据)
df9['EndDate'] = df9['EndDate'].astype('datetime64[ns]')

# 生成银行类型
from result_process_codes.daixiao_comp_basic_info import daixiao_comp_basic_info
底层数据_代销机构基本信息 = daixiao_comp_basic_info(start_date,end_date,df2,df8)
df1 = pd.merge(df1,底层数据_代销机构基本信息[['comp_type']],how='left',left_on='ParentCompName',right_index=True)

 *生成【底层数据_理财公司基本信息】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


In [4]:
#生成用于检查的数据
import dateutil
import numpy as np
import pandas as pd
from data_process_codes.preprocess import exclude_mother_child_relation, preprocess
import warnings
import empyrical as emp
import sys
date = end_date
month_begin_date = date - dateutil.relativedelta.relativedelta(months=1) + dateutil.relativedelta.relativedelta(days=1)#月初
df1_temp = preprocess(df1,month_begin_date)
df2_temp = df2[(df2['代销开始日']<date)&(df2['代销结束日']>month_begin_date)]
df3_temp = pd.merge(df1_temp,df2_temp,how='inner',left_on=['RegistrationCode','FinProCode'],right_on=['产品登记编码','普益代码'])#合并匹配基础数据和代销数据
df3_temp = df3_temp.drop(df3_temp [(df3_temp['MinInvestTimeType']=='数据缺失')|(df3_temp['MinInvestTimeType']=='其他')].index)#处理数据缺失的情况
df3_temp = df3_temp.dropna(subset=['MinInvestTimeType'])
df7_temp = df7.loc[:,df7.columns<date]
年化收益 = df7_temp.apply(lambda x:emp.annual_return((x/x.shift(1)-1).dropna(),annualization=52),axis=1).rename('年化收益').reset_index()
df3_temp = pd.merge(df3_temp,年化收益,how='left',on='FinProCode')
df3_temp.replace('其他半开型','封闭式',inplace = True)
df3_temp['发行机构_copy'] = df3_temp['发行机构'].copy()
df4_temp = exclude_mother_child_relation(df3_temp)#剔除母子公司之间建立的代销关系
# df3_temp.to_excel(r'C:\noahzhan\研究生\建投实习\代销图鉴\用于检查的数据_未剔除母子关系0803.xlsx')
# df4_temp.to_excel(r'C:\noahzhan\研究生\建投实习\代销图鉴\用于检查的数据_剔除母子关系0803.xlsx')

In [7]:
from daixiao_comp_netvalue_analysis import daixiao_comp_netvalue_analysis
from licai_comp_basic_info import licai_comp_basic_info
from daixiao_comp_basic_info import daixiao_comp_basic_info
from licai_comp_daixiaoproduct_info import licai_comp_daixiaoproduct_info
from daixiao_comp_daixiaoproduct_info import daixiao_comp_daixiaoproduct_info
from licai_comp_daixiaoproduct_info import licai_comp_daixiaoproduct_info_draw
from daixiao_comp_daixiaoproduct_info import daixiao_comp_daixiaoproduct_info_draw
from hezuoproduct_num import hezuoproduct_num
from licai_comp_netvalue_analysis import licai_comp_netvalue_analysis
from licai_comp_product_shelves_info import licai_comp_product_shelves_info
from daixiao_comp_product_shelves_info import daixiao_comp_product_shelves_info

from licai_comp_fixed_shalves import licai_comp_fixed_shalves
from daixiao_comp_fixed_shalves import daixiao_comp_fixed_shalves
from product_fees import product_fees
from licai_comp_gonggao_analysis import licai_comp_gonggao_analysis
from daixiao_comp_gonggao_analysis import daixiao_comp_gonggao_analysis
from cash_product_info import cash_product_info_draw
from cash_product_info import cash_product_info
from licai_comp_product_recommendation import licai_comp_product_recommendation
from daixiao_comp_product_recommendation import daixiao_comp_product_recommendation
#基本信息表
底层数据_理财公司基本信息 = licai_comp_basic_info(start_date,end_date,df1)
底层数据_代销机构基本信息 = daixiao_comp_basic_info(start_date,end_date,df2,df8)
#代销产品概况
底层数据_代销产品概况_理财公司 = pd.concat([licai_comp_daixiaoproduct_info(dt.datetime(2022,5,1),end_date_ts,df1,df2),#单机构数据
                                        licai_comp_daixiaoproduct_info(dt.datetime(2022,5,1),end_date_ts,df1,df2,result_type="licai_sector")],#板块数据
                                        axis=0)
底层数据_代销产品概况_代销机构 = pd.concat([daixiao_comp_daixiaoproduct_info(dt.datetime(2022,5,1),end_date_ts,df1,df2),#单机构数据
                                        daixiao_comp_daixiaoproduct_info(dt.datetime(2022,5,1),end_date_ts,df1,df2,result_type="daixiao_sector")],#板块数据
                                        axis=0)
底层数据_代销产品概况_理财公司_temp = 底层数据_代销产品概况_理财公司.copy()
底层数据_代销产品概况_代销机构_temp = 底层数据_代销产品概况_代销机构.copy()
底层数据_代销产品概况_理财公司_temp.loc[:,['准入代销机构排名','被代销产品数排名','产品被代销次数排名']] = 底层数据_代销产品概况_理财公司[['准入代销机构排名','被代销产品数排名','产品被代销次数排名']].apply(lambda x:x.apply(lambda y:float(y.split('/')[0])),axis=0)
底层数据_代销产品概况_理财公司_绘图 = licai_comp_daixiaoproduct_info_draw(dt.datetime(2022,5,1),end_date_ts,底层数据_代销产品概况_理财公司_temp)
底层数据_代销产品概况_代销机构_temp.loc[:,['准入管理人数排名','代销产品数量排名']] = 底层数据_代销产品概况_代销机构[['准入管理人数排名','代销产品数量排名']].apply(lambda x:x.apply(lambda y:float(y.split('/')[0])),axis=0)
底层数据_代销产品概况_代销机构_绘图 = daixiao_comp_daixiaoproduct_info_draw(dt.datetime(2022,5,1),end_date_ts,底层数据_代销产品概况_代销机构_temp)
#合作产品数
底层数据_合作产品 = pd.concat([hezuoproduct_num(start_date,end_date,df1,df2),#单机构数据
                            hezuoproduct_num(start_date,end_date,df1,df2,if_sector=True)],#板块数据
                            axis=0)
#代销产品货架
底层数据_代销产品货架_理财公司 = pd.concat([licai_comp_product_shelves_info(start_date_ts,end_date_ts,df1,df2,df7),
                                        licai_comp_product_shelves_info(start_date_ts,end_date_ts,df1,df2,df7,result_type='licai_sector')],
                                        axis=0)
底层数据_代销产品货架_代销机构 = pd.concat([daixiao_comp_product_shelves_info(start_date_ts,end_date_ts,df1,df2,df7),
                                        daixiao_comp_product_shelves_info(start_date_ts,end_date_ts,df1,df2,df7,result_type='daixiao_sector')],
                                        axis=0)
#固定收益类货架
底层数据_固定收益类货架_理财公司 = pd.concat([licai_comp_fixed_shalves(start_date,end_date,df1,df2,df7),
                                        licai_comp_fixed_shalves(start_date,end_date,df1,df2,df7,result_type='licai_sector')],
                                        axis=0)
底层数据_固定收益类货架_代销机构 = pd.concat([daixiao_comp_fixed_shalves(start_date,end_date,df1,df2,df7),
                                        daixiao_comp_fixed_shalves(start_date,end_date,df1,df2,df7,result_type='daixiao_sector')],
                                        axis=0)
#产品费率
底层数据_产品费率 = pd.concat([product_fees(start_date,end_date,df1,df2),
                            product_fees(start_date,end_date,df1,df2,if_sector=True)],
                            axis=0)
#公告分析
底层数据_公告分析_理财公司_single = licai_comp_gonggao_analysis(start_date,df1,df2,df7)
底层数据_公告分析_理财公司_sector = licai_comp_gonggao_analysis(start_date,df1,df2,df7,result_type='licai_sector')
底层数据_公告分析_理财公司=pd.concat([底层数据_公告分析_理财公司_single,底层数据_公告分析_理财公司_sector])
底层数据_公告分析_代销机构_single = daixiao_comp_gonggao_analysis(start_date,df1,df2,df7)
底层数据_公告分析_代销机构_sector = daixiao_comp_gonggao_analysis(start_date,df1,df2,df7,result_type='daixiao_sector')
底层数据_公告分析_代销机构=pd.concat([底层数据_公告分析_代销机构_single,底层数据_公告分析_代销机构_sector])
#净值分析
底层数据_净值指标_理财公司 = pd.concat([licai_comp_netvalue_analysis(start_date,df1,df2,df7,result_type='single'),
        licai_comp_netvalue_analysis(start_date,df1,df2,df7,result_type='licai_sector')],
        axis=0)
底层数据_净值指标_代销机构 = pd.concat([daixiao_comp_netvalue_analysis(start_date,df1,df2,df7,result_type='single'),
        daixiao_comp_netvalue_analysis(start_date,df1,df2,df7,result_type='daixiao_sector')],
        axis=0)
#现金管理类产品分析
底层数据_现金管理类产品七日年化收益率作图_single = cash_product_info_draw(start_date,end_date,df1,df2,df9,if_sector=False)
底层数据_现金管理类产品七日年化收益率作图_sector = cash_product_info_draw(start_date,end_date,df1,df2,df9,if_sector=True)
底层数据_现金管理类产品七日年化收益率作图 = pd.concat([底层数据_现金管理类产品七日年化收益率作图_single.iloc[:,:-1],
                                                底层数据_现金管理类产品七日年化收益率作图_sector],
                                                axis=1)
底层数据_现金管理类产品分析 = pd.concat([cash_product_info(底层数据_现金管理类产品七日年化收益率作图_single.iloc[:,:-1]),
                                    cash_product_info(底层数据_现金管理类产品七日年化收益率作图_sector.iloc[:,:-1])],
                                    axis=0)
#产品甄选
底层数据_产品甄选_理财公司_近一年,底层数据_产品甄选_理财公司_近半年,底层数据_产品甄选_理财公司_近三月 = licai_comp_product_recommendation(start_date,end_date,df1,df2,df7,df9)
底层数据_产品甄选_理财公司 = pd.concat([底层数据_产品甄选_理财公司_近一年,底层数据_产品甄选_理财公司_近半年.reset_index(),底层数据_产品甄选_理财公司_近三月.reset_index()],axis=1)
底层数据_产品甄选_理财公司_近一年_板块,底层数据_产品甄选_理财公司_近半年_板块,底层数据_产品甄选_理财公司_近三月_板块 = licai_comp_product_recommendation(start_date,end_date,df1,df2,df7,df9,result_type='licai_sector')
底层数据_产品甄选_理财公司_板块 = pd.concat([底层数据_产品甄选_理财公司_近一年_板块,底层数据_产品甄选_理财公司_近半年_板块.reset_index(),底层数据_产品甄选_理财公司_近三月_板块.reset_index()],axis=1)
底层数据_产品甄选_代销机构_近一年,底层数据_产品甄选_代销机构_近半年,底层数据_产品甄选_代销机构_近三月 = daixiao_comp_product_recommendation(start_date,end_date,df1,df2,df7,df9)
底层数据_产品甄选_代销机构 = pd.concat([底层数据_产品甄选_代销机构_近一年,底层数据_产品甄选_代销机构_近半年.reset_index(),底层数据_产品甄选_代销机构_近三月.reset_index()],axis=1)
底层数据_产品甄选_代销机构_近一年_板块,底层数据_产品甄选_代销机构_近半年_板块,底层数据_产品甄选_代销机构_近三月_板块 = daixiao_comp_product_recommendation(start_date,end_date,df1,df2,df7,df9,result_type='daixiao_sector')
底层数据_产品甄选_代销机构_板块 = pd.concat([底层数据_产品甄选_代销机构_近一年_板块,底层数据_产品甄选_代销机构_近半年_板块.reset_index(),底层数据_产品甄选_代销机构_近三月_板块.reset_index()],axis=1)

#输出
底层数据_理财公司基本信息.to_excel(path_outputdir + "/底层数据_理财公司基本信息.xlsx")
底层数据_代销机构基本信息.to_excel(path_outputdir + "/底层数据_代销机构基本信息.xlsx")
底层数据_代销产品概况_理财公司.to_excel(path_outputdir + "/底层数据_代销产品概况_理财公司.xlsx")
底层数据_代销产品概况_代销机构.to_excel(path_outputdir + "/底层数据_代销产品概况_代销机构.xlsx")
底层数据_代销产品概况_理财公司_绘图.to_excel(path_outputdir + "/底层数据_代销产品概况_理财公司_绘图.xlsx")
底层数据_代销产品概况_代销机构_绘图.to_excel(path_outputdir + "/底层数据_代销产品概况_代销机构_绘图.xlsx")
底层数据_合作产品.to_excel(path_outputdir + "/底层数据_合作产品.xlsx")
底层数据_代销产品货架_理财公司.to_excel(path_outputdir + "/底层数据_代销产品货架_理财公司.xlsx")
底层数据_代销产品货架_代销机构.to_excel(path_outputdir + "/底层数据_代销产品货架_代销机构.xlsx")
底层数据_固定收益类货架_理财公司.to_excel(path_outputdir + "/底层数据_固定收益类货架_理财公司.xlsx")
底层数据_固定收益类货架_代销机构.to_excel(path_outputdir + "/底层数据_固定收益类货架_代销机构.xlsx")
底层数据_产品费率.to_excel(path_outputdir + "/底层数据_产品费率.xlsx")
底层数据_公告分析_理财公司.to_excel(path_outputdir + "/底层数据_公告分析_理财公司.xlsx")
底层数据_公告分析_代销机构.to_excel(path_outputdir + "/底层数据_公告分析_代销机构.xlsx")
底层数据_净值指标_理财公司.to_excel(path_outputdir + "/底层数据_净值指标_理财公司.xlsx")
底层数据_净值指标_代销机构.to_excel(path_outputdir + "/底层数据_净值指标_代销机构.xlsx")
底层数据_现金管理类产品七日年化收益率作图.to_excel(path_outputdir + "/底层数据_现金管理类产品七日年化收益率作图.xlsx")
底层数据_现金管理类产品分析.to_excel(path_outputdir + "/底层数据_现金管理类产品分析.xlsx")
底层数据_产品甄选_理财公司.to_excel(path_outputdir + "/底层数据_产品甄选_理财公司.xlsx")
底层数据_产品甄选_理财公司_板块.to_excel(path_outputdir + "/底层数据_产品甄选_理财公司_板块.xlsx")
底层数据_产品甄选_代销机构.to_excel(path_outputdir + "/底层数据_产品甄选_代销机构.xlsx")
底层数据_产品甄选_代销机构_板块.to_excel(path_outputdir + "/底层数据_产品甄选_代销机构_板块.xlsx")

 *生成【底层数据_理财公司基本信息】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


c:\Users\15620\.conda\envs\python39\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


 *生成【底层数据_理财公司基本信息】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 *生成【底层数据_代销产品概况_理财公司】依赖表,  2022-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


c:\Users\15620\.conda\envs\python39\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


 -Processing date  2023-04-30 00:00:00


...

c:\Users\15620\.conda\envs\python39\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\15620\.conda\envs\python39\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\15620\.conda\envs\python39\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated Nu

 -Processing date  2023-05-31 00:00:00


c:\Users\15620\.conda\envs\python39\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\15620\.conda\envs\python39\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\15620\.conda\envs\python39\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated Nu

 -Processing date  2023-05-31 00:00:00


c:\Users\15620\.conda\envs\python39\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\15620\.conda\envs\python39\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\15620\.conda\envs\python39\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated Nu

In [23]:
#底层数据_理财公司基本信息
from result_process_codes.licai_comp_basic_info import licai_comp_basic_info
底层数据_理财公司基本信息_for_lc_section = licai_comp_basic_info(start_date,end_date,df1)
# 底层数据_理财公司基本信息.to_excel(path_outputdir+'/底层数据_理财公司基本信息.xlsx')
底层数据_理财公司基本信息_for_lc_section
#底层数据_代销机构基本信息
from result_process_codes.daixiao_comp_basic_info import daixiao_comp_basic_info
底层数据_代销机构基本信息 = daixiao_comp_basic_info(start_date,end_date,df2,df8)
# 底层数据_代销机构基本信息.to_excel(path_outputdir+'/底层数据_代销机构基本信息.xlsx')
底层数据_代销机构基本信息

 *生成【底层数据_理财公司基本信息】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 *生成【底层数据_理财公司基本信息】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


,comp_name,comp_simple_name,comp_type,found_date,regcapital,province,代销机构,代销产品数量,代销产品数量排名,规模排名,日期
代销机构,,,,,,,,,,,
宜宾市商业银行,宜宾市商业银行,宜宾市商业银行,城市商业银行,20061227.0,3.900000e+05,四川省,宜宾市商业银行,1,419/432,83.0/432,2023-05-31
浙江建德湖商村镇银行,浙江建德湖商村镇银行,浙江建德湖商村镇银行,村镇银行,20091111.0,1.500000e+04,浙江省,浙江建德湖商村镇银行,1,419/432,393.0/432,2023-05-31
江苏泰州农村商业银行,江苏泰州农村商业银行,江苏泰州农商行,农商行,20070412.0,1.080216e+05,江苏省,江苏泰州农村商业银行,1,419/432,182.0/432,2023-05-31
平顶山银行,平顶山银行,平顶山银行,城市商业银行,20011231.0,4.105983e+05,河南省,平顶山银行,1,419/432,75.0/432,2023-05-31
浙江民泰商业银行,浙江民泰商业银行,浙江民泰商业银行,城市商业银行,19880512.0,4.607289e+05,浙江省,浙江民泰商业银行,1,419/432,70.0/432,2023-05-31
...,...,...,...,...,...,...,...,...,...,...,...
江苏银行,江苏银行,江苏银行,城市商业银行,20070122.0,1.476968e+06,江苏省,江苏银行,1414,5/432,26.0/432,2023-05-31
招商银行,招商银行,招商银行,股份制商业银行,19870331.0,2.521985e+06,广东省,招商银行,1630,4/432,13.0/432,2023-05-31
中国银行,中国银行,中国银行,国有商业银行,19831031.0,2.943878e+07,北京,中国银行,1665,3/432,3.0/432,2023-05-31


In [5]:
#底层数据_理财公司基本信息
from result_process_codes.licai_comp_basic_info import licai_comp_basic_info
底层数据_理财公司基本信息 = licai_comp_basic_info(start_date,end_date,df1)
底层数据_理财公司基本信息.to_excel(path_outputdir+'/底层数据_理财公司基本信息.xlsx')
底层数据_理财公司基本信息

 *生成【底层数据_理财公司基本信息】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


,日期,理财公司,所属分类,产品数量,成立日期,理财产品存续总规模,规模排名
理财公司简称,,,,,,,
上银理财,2023-05-31,上银理财,城商行全资理财子公司,361,2022-03-15,3.445024e+11,18.0
中邮理财,2023-05-31,中邮理财,国有行全资理财子公司,478,2019-12-18,8.387969e+11,11.0
中银理财,2023-05-31,中银理财,国有行全资理财子公司,1131,2019-07-01,1.458895e+12,4.0
交银理财,2023-05-31,交银理财,国有行全资理财子公司,665,2019-06-06,1.141203e+12,9.0
信银理财,2023-05-31,信银理财,股份制行全资理财子公司,1446,2020-07-01,1.499735e+12,3.0
光大理财,2023-05-31,光大理财,股份制行全资理财子公司,447,2019-09-25,1.161361e+12,8.0
兴银理财,2023-05-31,兴银理财,股份制行全资理财子公司,838,2019-12-13,2.019947e+12,2.0
农银理财,2023-05-31,农银理财,国有行全资理财子公司,531,2019-07-29,1.415575e+12,5.0
北银理财,2023-05-31,北银理财,城商行全资理财子公司,212,2022-11-25,2.761309e+11,20.0


In [6]:
#底层数据_代销机构基本信息
from result_process_codes.daixiao_comp_basic_info import daixiao_comp_basic_info
底层数据_代销机构基本信息 = daixiao_comp_basic_info(start_date,end_date,df2,df8)
底层数据_代销机构基本信息.to_excel(path_outputdir+'/底层数据_代销机构基本信息.xlsx')

 *生成【底层数据_理财公司基本信息】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


In [47]:
#底层数据_代销产品概况_理财公司
from licai_comp_daixiaoproduct_info import licai_comp_daixiaoproduct_info
底层数据_代销产品概况_理财公司 = licai_comp_daixiaoproduct_info(dt.datetime(2022,5,1),end_date_ts,df1,df2)
底层数据_代销产品概况_理财公司.to_excel(path_outputdir+'/底层数据_代销产品概况_理财公司.xlsx')
底层数据_代销产品概况_理财公司_板块 = licai_comp_daixiaoproduct_info(dt.datetime(2022,5,1),end_date_ts,df1,df2,result_type="licai_sector")
底层数据_代销产品概况_理财公司_板块.to_excel(path_outputdir+'/底层数据_代销产品概况_理财公司_板块.xlsx')

 *生成【底层数据_代销产品概况_理财公司】依赖表,  2022-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-04-30 00:00:00
 -Processing date  2023-03-31 00:00:00
 -Processing date  2023-02-28 00:00:00
 -Processing date  2023-01-31 00:00:00
 -Processing date  2022-12-31 00:00:00
 -Processing date  2022-11-30 00:00:00
 -Processing date  2022-10-31 00:00:00
 -Processing date  2022-09-30 00:00:00
 -Processing date  2022-08-31 00:00:00
 -Processing date  2022-07-31 00:00:00
 -Processing date  2022-06-30 00:00:00
 -Processing date  2022-05-31 00:00:00
 *生成【底层数据_代销产品概况_理财公司】依赖表,  2022-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-04-30 00:00:00
 -Processing date  2023-03-31 00:00:00
 -Processing date  2023-02-28 00:00:00
 -Processing date  2023-01-31 00:00:00
 -Processing date  2022-12-31 00:00:00
 -Processing date  2022-11-30 00:00:00
 -Processing date  2022-10-31 00:00:00
 -Processing date  2022-09-30 00:

In [48]:
#底层数据_代销产品概况_代销机构
from daixiao_comp_daixiaoproduct_info import daixiao_comp_daixiaoproduct_info
底层数据_代销产品概况_代销机构 = daixiao_comp_daixiaoproduct_info(dt.datetime(2022,5,1),end_date_ts,df1,df2)
底层数据_代销产品概况_代销机构.to_excel(path_outputdir+'/底层数据_代销产品概况_代销机构.xlsx')
底层数据_代销产品概况_代销机构_板块 = daixiao_comp_daixiaoproduct_info(dt.datetime(2022,5,1),end_date_ts,df1,df2,result_type="daixiao_sector")
底层数据_代销产品概况_代销机构_板块.to_excel(path_outputdir+'/底层数据_代销产品概况_代销机构_板块.xlsx')

 *生成【底层数据_代销产品概况_代销机构】依赖表,  2022-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-04-30 00:00:00
 -Processing date  2023-03-31 00:00:00
 -Processing date  2023-02-28 00:00:00
 -Processing date  2023-01-31 00:00:00
 -Processing date  2022-12-31 00:00:00
 -Processing date  2022-11-30 00:00:00
 -Processing date  2022-10-31 00:00:00
 -Processing date  2022-09-30 00:00:00
 -Processing date  2022-08-31 00:00:00
 -Processing date  2022-07-31 00:00:00
 -Processing date  2022-06-30 00:00:00
 -Processing date  2022-05-31 00:00:00
 *生成【底层数据_代销产品概况_代销机构】依赖表,  2022-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-04-30 00:00:00
 -Processing date  2023-03-31 00:00:00
 -Processing date  2023-02-28 00:00:00
 -Processing date  2023-01-31 00:00:00
 -Processing date  2022-12-31 00:00:00
 -Processing date  2022-11-30 00:00:00
 -Processing date  2022-10-31 00:00:00
 -Processing date  2022-09-30 00:

In [49]:
#底层数据_代销产品概况_理财公司_绘图
from licai_comp_daixiaoproduct_info import licai_comp_daixiaoproduct_info_draw
底层数据_代销产品概况_理财公司.loc[:,['准入代销机构排名','被代销产品数排名','产品被代销次数排名']] = 底层数据_代销产品概况_理财公司[['准入代销机构排名','被代销产品数排名','产品被代销次数排名']].apply(lambda x:x.apply(lambda y:float(y.split('/')[0])),axis=0)
底层数据_代销产品概况_理财公司_绘图 = licai_comp_daixiaoproduct_info_draw(dt.datetime(2022,5,1),end_date_ts,底层数据_代销产品概况_理财公司)
底层数据_代销产品概况_理财公司_绘图.to_excel(path_outputdir+'/底层数据_代销产品概况_理财公司_绘图.xlsx')
底层数据_代销产品概况_理财公司_板块.loc[:,['准入代销机构排名','被代销产品数排名','产品被代销次数排名']] = 底层数据_代销产品概况_理财公司_板块[['准入代销机构排名','被代销产品数排名','产品被代销次数排名']].apply(lambda x:x.apply(lambda y:float(y.split('/')[0])),axis=0)
底层数据_代销产品概况_理财公司_绘图_板块 = licai_comp_daixiaoproduct_info_draw(dt.datetime(2022,5,1),end_date_ts,底层数据_代销产品概况_理财公司_板块)
底层数据_代销产品概况_理财公司_绘图_板块.to_excel(path_outputdir+'/底层数据_代销产品概况_理财公司_绘图_板块.xlsx')

 *生成【底层数据_代销产品概况_理财公司_绘图】依赖表,  2022-05-01 00:00:00 到 2023-05-31 00:00:00 ...
processing date: 2023-05-31 00:00:00
processing date: 2023-04-30 00:00:00
processing date: 2023-03-31 00:00:00
processing date: 2023-02-28 00:00:00
processing date: 2023-01-31 00:00:00
processing date: 2022-12-31 00:00:00
processing date: 2022-11-30 00:00:00
processing date: 2022-10-31 00:00:00
processing date: 2022-09-30 00:00:00
processing date: 2022-08-31 00:00:00
processing date: 2022-07-31 00:00:00
processing date: 2022-06-30 00:00:00
processing date: 2022-05-31 00:00:00
 *生成【底层数据_代销产品概况_理财公司_绘图】依赖表,  2022-05-01 00:00:00 到 2023-05-31 00:00:00 ...
processing date: 2023-05-31 00:00:00
processing date: 2023-04-30 00:00:00
processing date: 2023-03-31 00:00:00
processing date: 2023-02-28 00:00:00
processing date: 2023-01-31 00:00:00
processing date: 2022-12-31 00:00:00
processing date: 2022-11-30 00:00:00
processing date: 2022-10-31 00:00:00
processing date: 2022-09-30 00:00:00
processing date: 2022-08-31 00:0

In [50]:
#底层数据_代销产品概况_代销机构_绘图
from daixiao_comp_daixiaoproduct_info import daixiao_comp_daixiaoproduct_info_draw
底层数据_代销产品概况_代销机构.loc[:,['准入管理人数排名','代销产品数量排名']] = 底层数据_代销产品概况_代销机构[['准入管理人数排名','代销产品数量排名']].apply(lambda x:x.apply(lambda y:float(y.split('/')[0])),axis=0)
底层数据_代销产品概况_代销机构_绘图 = daixiao_comp_daixiaoproduct_info_draw(dt.datetime(2022,5,1),end_date_ts,底层数据_代销产品概况_代销机构)
底层数据_代销产品概况_代销机构_绘图.to_excel(path_outputdir+'/底层数据_代销产品概况_代销机构_绘图.xlsx')
底层数据_代销产品概况_代销机构_板块.loc[:,['准入管理人数排名','代销产品数量排名']] = 底层数据_代销产品概况_代销机构_板块[['准入管理人数排名','代销产品数量排名']].apply(lambda x:x.apply(lambda y:float(y.split('/')[0])),axis=0)
底层数据_代销产品概况_代销机构_绘图_板块 = daixiao_comp_daixiaoproduct_info_draw(dt.datetime(2022,5,1),end_date_ts,底层数据_代销产品概况_代销机构_板块)
底层数据_代销产品概况_代销机构_绘图_板块.to_excel(path_outputdir+'/底层数据_代销产品概况_代销机构_绘图_板块.xlsx')

 *生成【底层数据_代销产品概况_代销机构_绘图】依赖表,  2022-05-01 00:00:00 到 2023-05-31 00:00:00 ...
processing date: 2023-05-31 00:00:00
processing date: 2023-04-30 00:00:00
processing date: 2023-03-31 00:00:00
processing date: 2023-02-28 00:00:00
processing date: 2023-01-31 00:00:00
processing date: 2022-12-31 00:00:00
processing date: 2022-11-30 00:00:00
processing date: 2022-10-31 00:00:00
processing date: 2022-09-30 00:00:00
processing date: 2022-08-31 00:00:00
processing date: 2022-07-31 00:00:00
processing date: 2022-06-30 00:00:00
processing date: 2022-05-31 00:00:00
 *生成【底层数据_代销产品概况_代销机构_绘图】依赖表,  2022-05-01 00:00:00 到 2023-05-31 00:00:00 ...
processing date: 2023-05-31 00:00:00
processing date: 2023-04-30 00:00:00
processing date: 2023-03-31 00:00:00
processing date: 2023-02-28 00:00:00
processing date: 2023-01-31 00:00:00
processing date: 2022-12-31 00:00:00
processing date: 2022-11-30 00:00:00
processing date: 2022-10-31 00:00:00
processing date: 2022-09-30 00:00:00
processing date: 2022-08-31 00:0

In [10]:
# #底层数据_合作产品
from hezuoproduct_num import hezuoproduct_num
底层数据_合作产品 = hezuoproduct_num(start_date,end_date,df1,df2)
底层数据_合作产品.to_excel(path_outputdir+'/底层数据_合作产品.xlsx')
底层数据_合作产品_板块 = hezuoproduct_num(start_date,end_date,df1,df2,if_sector=True)
底层数据_合作产品_板块.to_excel(path_outputdir+'/底层数据_合作产品_板块.xlsx')

 *生成【底层数据_合作产品】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 *生成【底层数据_合作产品】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


In [11]:
#底层数据_代销产品货架_理财公司
from licai_comp_product_shelves_info import licai_comp_product_shelves_info
底层数据_代销产品货架_理财公司 = licai_comp_product_shelves_info(start_date,end_date,df1,df2,df7)
底层数据_代销产品货架_理财公司.to_excel(path_outputdir+'/底层数据_代销产品货架_理财公司.xlsx')
底层数据_代销产品货架_理财公司_板块 = licai_comp_product_shelves_info(start_date,end_date,df1,df2,df7,result_type='licai_sector')
底层数据_代销产品货架_理财公司_板块.to_excel(path_outputdir+'/底层数据_代销产品货架_理财公司_板块.xlsx')

 *生成【底层数据_代销产品概况_理财公司】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 *生成【底层数据_代销产品概况_理财公司】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


In [12]:
#底层数据_代销产品货架_代销机构
from daixiao_comp_product_shelves_info import daixiao_comp_product_shelves_info
底层数据_代销产品货架_代销机构 = daixiao_comp_product_shelves_info(start_date,end_date,df1,df2,df7)
底层数据_代销产品货架_代销机构.to_excel(path_outputdir+'/底层数据_代销产品货架_代销机构.xlsx')
底底层数据_代销产品货架_代销机构_板块 = daixiao_comp_product_shelves_info(start_date,end_date,df1,df2,df7,result_type='daixiao_sector')
底底层数据_代销产品货架_代销机构_板块.to_excel(path_outputdir+'/底底层数据_代销产品货架_代销机构_板块.xlsx')

 *生成【底层数据_代销产品货架_代销机构】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 *生成【底层数据_代销产品货架_代销机构】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


In [13]:
# 底层数据_固定收益类货架_理财公司
from licai_comp_fixed_shalves import licai_comp_fixed_shalves
底层数据_固定收益类货架_理财公司 = licai_comp_fixed_shalves(start_date,end_date,df1,df2,df7)
底层数据_固定收益类货架_理财公司.to_excel(path_outputdir+'/底层数据_固定收益类货架_理财公司.xlsx')
底层数据_固定收益类货架_理财公司_板块 = licai_comp_fixed_shalves(start_date,end_date,df1,df2,df7,result_type='licai_sector')
底层数据_固定收益类货架_理财公司_板块.to_excel(path_outputdir+'/底层数据_固定收益类货架_理财公司_板块.xlsx')

 *生成【 底层数据_固定收益类货架_理财公司】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 *生成【 底层数据_固定收益类货架_理财公司】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


In [5]:
# 底层数据_固定收益类货架_代销机构
from daixiao_comp_fixed_shalves import daixiao_comp_fixed_shalves
底层数据_固定收益类货架_代销机构 = daixiao_comp_fixed_shalves(start_date,end_date,df1,df2,df7)
底层数据_固定收益类货架_代销机构.to_excel(path_outputdir+'/底层数据_固定收益类货架_代销机构.xlsx')
底层数据_固定收益类货架_代销机构_板块 = daixiao_comp_fixed_shalves(start_date,end_date,df1,df2,df7,result_type='daixiao_sector')
底层数据_固定收益类货架_代销机构_板块.to_excel(path_outputdir+'/底层数据_固定收益类货架_代销机构_板块.xlsx')

 *生成【底层数据_固定收益类货架_代销机构】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 *生成【底层数据_固定收益类货架_代销机构】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


In [15]:
# 底层数据_产品费率
from product_fees import product_fees
底层数据_产品费率 = product_fees(start_date,end_date,df1,df2)
底层数据_产品费率.to_excel(path_outputdir+'/底层数据_产品费率.xlsx')
底层数据_产品费率_板块 = product_fees(start_date,end_date,df1,df2,if_sector=True)
底层数据_产品费率_板块.to_excel(path_outputdir+'/底层数据_产品费率_板块.xlsx')

 *生成【底层数据_产品费率】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00
 *生成【底层数据_产品费率】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...
 -Processing date  2023-05-31 00:00:00


In [70]:
# 底层数据_现金管理类产品七日年化收益率作图 = cash_product_info_draw(start_date,end_date,df1,df2,df7,df9,if_sector=True)
底层数据_现金管理类产品分析 = cash_product_info(底层数据_现金管理类产品七日年化收益率作图)
底层数据_现金管理类产品分析#.to_excel(r'C:\noahzhan\研究生\建投实习\代销图鉴\银行理财代销图鉴_code_0724\test.xlsx')

发行机构: 100%|██████████| 4/4 [00:01<00:00,  2.36it/s]


,近一周年化收益率,近一月年化收益率,近三月年化收益率,近半年年化收益率,近一年年化收益率,近一周年化收益率_排名,近三月年化收益率_排名,近半年年化收益率_排名,近一年年化收益率_排名,机构类型,是否剔除母子关系
农村金融机构全资理财子公司,0.026197,0.026045,0.025568,0.025655,NaN,1.0/4,1.0/4,1.0/4,nan/3,发行机构,是
国有行全资理财子公司,0.022813,0.020350,0.019920,0.019870,0.020061,4.0/4,4.0/4,4.0/4,3.0/3,发行机构,是
城商行全资理财子公司,0.024338,0.023325,0.023237,0.022668,0.023097,2.0/4,2.0/4,3.0/4,1.0/3,发行机构,是
股份制行全资理财子公司,0.024611,0.023080,0.022845,0.022709,0.022819,3.0/4,3.0/4,2.0/4,2.0/3,发行机构,是
农商行,0.026197,0.026045,0.025568,0.025655,NaN,1.0/4,1.0/4,1.0/4,nan/3,代销机构,是
国有商业银行,0.022813,0.020350,0.019920,0.019870,0.020061,4.0/4,4.0/4,4.0/4,3.0/3,代销机构,是
城市商业银行,0.024338,0.023325,0.023237,0.022668,0.023097,2.0/4,2.0/4,3.0/4,1.0/3,代销机构,是
股份制商业银行,0.024611,0.023080,0.022845,0.022709,0.022819,3.0/4,3.0/4,2.0/4,2.0/3,代销机构,是
农村金融机构全资理财子公司,0.021395,0.021357,0.021191,0.020189,0.021002,3.0/4,3.0/4,3.0/4,3.0/4,发行机构,否
国有行全资理财子公司,0.021304,0.020023,0.019808,0.019514,0.019850,4.0/4,4.0/4,4.0/4,4.0/4,发行机构,否


In [22]:
#底层数据_产品甄选_理财公司
from licai_comp_product_recommendation import licai_comp_product_recommendation
底层数据_产品甄选_理财公司_近一年,底层数据_产品甄选_理财公司_近半年,底层数据_产品甄选_理财公司_近三月 = licai_comp_product_recommendation(start_date,end_date,df1,df2,df7,df9)
底层数据_产品甄选_理财公司 = pd.concat([底层数据_产品甄选_理财公司_近一年,底层数据_产品甄选_理财公司_近半年.reset_index(),底层数据_产品甄选_理财公司_近三月.reset_index()],axis=1)
底层数据_产品甄选_理财公司.to_excel(path_outputdir+'/底层数据_产品甄选_理财公司.xlsx')
底层数据_产品甄选_理财公司_近一年_板块,底层数据_产品甄选_理财公司_近半年_板块,底层数据_产品甄选_理财公司_近三月_板块 = licai_comp_product_recommendation(start_date,end_date,df1,df2,df7,df9,result_type='licai_sector')
底层数据_产品甄选_理财公司_板块 = pd.concat([底层数据_产品甄选_理财公司_近一年_板块,底层数据_产品甄选_理财公司_近半年_板块.reset_index(),底层数据_产品甄选_理财公司_近三月_板块.reset_index()],axis=1)
底层数据_产品甄选_理财公司_板块.to_excel(path_outputdir+'/底层数据_产品甄选_理财公司_板块.xlsx')

 *生成【底层数据_产品甄选_理财公司】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...


C:\Users\53497\AppData\Roaming\Python\Python39\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(


 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-05-31 00:00:00
 *生成【底层数据_产品甄选_理财公司】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...


C:\Users\53497\AppData\Roaming\Python\Python39\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(


 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-05-31 00:00:00


In [23]:
#底层数据_产品甄选_代销机构
from daixiao_comp_product_recommendation import daixiao_comp_product_recommendation
底层数据_产品甄选_代销机构_近一年,底层数据_产品甄选_代销机构_近半年,底层数据_产品甄选_代销机构_近三月 = daixiao_comp_product_recommendation(start_date,end_date,df1,df2,df7,df9)
底层数据_产品甄选_代销机构 = pd.concat([底层数据_产品甄选_代销机构_近一年,底层数据_产品甄选_代销机构_近半年.reset_index(),底层数据_产品甄选_代销机构_近三月.reset_index()],axis=1)
底层数据_产品甄选_代销机构.to_excel(path_outputdir+'/底层数据_产品甄选_代销机构.xlsx')
底层数据_产品甄选_代销机构_近一年_板块,底层数据_产品甄选_代销机构_近半年_板块,底层数据_产品甄选_代销机构_近三月_板块 = daixiao_comp_product_recommendation(start_date,end_date,df1,df2,df7,df9,result_type='daixiao_sector')
底层数据_产品甄选_代销机构_板块 = pd.concat([底层数据_产品甄选_代销机构_近一年_板块,底层数据_产品甄选_代销机构_近半年_板块.reset_index(),底层数据_产品甄选_代销机构_近三月_板块.reset_index()],axis=1)
底层数据_产品甄选_代销机构_板块.to_excel(path_outputdir+'/底层数据_产品甄选_代销机构_板块.xlsx')

 *生成【底层数据_产品甄选_代销机构】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...


C:\Users\53497\AppData\Roaming\Python\Python39\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(


 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-05-31 00:00:00
 *生成【底层数据_产品甄选_代销机构】依赖表,  2023-05-01 00:00:00 到 2023-05-31 00:00:00 ...


C:\Users\53497\AppData\Roaming\Python\Python39\site-packages\empyrical\stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(


 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-05-31 00:00:00
 -Processing date  2023-05-31 00:00:00
